In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Gradio 應用程式和 Vertex AI 進行影像生成

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/vision/gradio/gradio_image_generation_sdk.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/vision/gradio/gradio_image_generation_sdk.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/vision/gradio/gradio_image_generation_sdk.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者| [Jose Brache](https://github.com/jbrache) |


## 簡介

此 Notebook 將建立 Gradio 應用程式 (前端)，透過整合 Vertex AI 中的 Imagen，使用自然語言提示產生高品質圖片。

此 Notebook 僅關注 Imagen 中的「圖片產生」功能。請注意，[圖片產生](https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview#feature-launch-stage) 目前處於**受限一般公開 (已核准使用者)** 階段。如欲使用 API，你需要在「Vertex AI Studio → Vision」下「產生」標籤的請求表單，透過 [Google Cloud 控制台](https://console.cloud.google.com/vertex-ai/generative/vision) 提出存取請求。

如需進一步瞭解如何為圖片產生撰寫文字提示，請參閱 [提示指南](https://cloud.google.com/vertex-ai/docs/generative-ai/image/img-gen-prompt-guide) 和下列資源：
- 在產生圖片時，有許多標準和進階 [參數](https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images#use-params) 可以根據你的使用案例設定。
- 你可以使用各種版本的 `imagegeneration` 模型。如需 Imagen 模型版本說明的一般資訊，請參閱 [官方文件](https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images#model-versions)。

你可以透過 [Google Cloud 控制台](https://console.cloud.google.com/vertex-ai/generative/vision) 或呼叫 Vertex AI API 來存取 Imagen。可在 [官方文件](https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images) 中找到 Vertex AI 中 Imagen 圖片產生的更進一步資訊。


### 目標

在本筆記本中，你將學習如何：

- 使用 [Imagen 2](https://cloud.google.com/blog/products/ai-machine-learning/imagen-2-on-vertex-ai-is-now-generally-available) (`imagegeneration@005`) 與 Vertex AI SDK，從文字提示產生新圖片

- 使用不同的參數進行實驗，例如：
    - 使用範例或自己的文字提示來產生圖片
    - 產生圖片的模型版本
    - 提供種子，根據輸入重現相同的圖片輸出

- 啟動 [Gradio 應用程式](https://www.gradio.app/) 以存取 Imagen


### 費用

- 本筆記本使用 Google Cloud 的計費元件：
  - Vertex AI (Imagen) 

- 進一步了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)，並使用 [費率計算器](https://cloud.google.com/products/calculator/) 依據你的預計使用狀況產生成本估計。


## 開始使用


### 安裝 Vertex AI SDK 及其他套裝及其相依套件

[Gradio](https://pypi.org/project/gradio/) 用於使用使用者介面互動使用 Imagen，測試時使用的版本為 `gradio==4.11.0` 和 `google-cloud-aiplatform==1.38.1`。


In [ ]:
!pip3 install --upgrade --user gradio
!pip3 install --upgrade --user google-cloud-aiplatform

### 重新啟動當前運行時

若要在這個 Jupyter 運行時中使用新安裝的套件，你必須重新啟動運行時。你可以執行下方的Cell來這麼做，這會重新啟動目前的 kernel。

重新啟動流程可能會花費一分鐘左右。


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 這個 kernel 將會重新啟動。請等到重新啟動程序完成後再進行下一步。⚠️</b>
</div>


### 認證你的 notebook 環境 (僅限 Colab) 

如果你是在 Google Colab 上執行這個筆記本，你將需要認證你的環境。為執行這項工作，請執行下列新的Cell。如果你使用的是 [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)，則不需要執行這個步驟。


In [ ]:
import sys

# Addtional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### 定義 Google Cloud 專案資訊 (僅適用於 Colab) 

如果你是在 Google Colab 上執行這個筆記本，你需要定義要使用的 Google Cloud 專案資訊。在以下Cell中，你會定義資訊、匯入 Vertex AI 套件並初始化它。如果你使用 [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)，也不需要進行這個步驟。


In [ ]:
if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

### 匯入函式庫


In [ ]:
import gradio as gr

from vertexai.preview.vision_models import Image, ImageGenerationModel

import traceback

# Gradio 應用程式

[Imagen 2](https://cloud.google.com/blog/products/ai-machine-learning/imagen-2-on-vertex-ai-is-now-generally-available) (`imagegeneration@005`) 是設計為從自然語言提示產生高品質、逼真、高解析度、美觀的圖像。

此區段會將 Imagen 的文字轉圖像生成功能包裝到 [Gradio 應用程式](https://www.gradio.app/docs/interface) 中，以使用範例提示進行互動式使用。Imagen 有支援不同功能的 [模型版本](https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images)，請參閱 [Imagen 在 Vertex AI 的模型版本和生命週期](https://cloud.google.com/vertex-ai/docs/generative-ai/image/model-versioning) 以取得更多資訊。


### 定義輔助函式

定義 Gradio 與 Vertex AI SDK 的輔助函式，用於載入和顯示圖片。


In [ ]:
# @title Helper functions
# Wrapper around the Vertex AI SDK to return PIL images
def imagen_generate(
    model_name: str,
    prompt: str,
    negative_prompt: str,
    sampleImageSize: int,
    sampleCount: int,
    seed=None,
):
    model = ImageGenerationModel.from_pretrained(model_name)

    generate_response = model.generate_images(
        prompt=prompt,
        negative_prompt=negative_prompt,
        number_of_images=sampleCount,
        guidance_scale=float(sampleImageSize),
        seed=seed,
    )

    images = []
    for index, result in enumerate(generate_response):
        images.append(generate_response[index]._pil_image)
    return images, generate_response


# Update function called by Gradio
def update(
    model_name,
    prompt,
    negative_prompt,
    sampleImageSize="1536",
    sampleCount=4,
    seed=None,
):
    if len(negative_prompt) == 0:
        negative_prompt = None

    print("prompt:", prompt)
    print("negative_prompt:", negative_prompt)

    # Advanced option, try different the seed numbers
    # any random integer number range: (0, 2147483647)
    if seed < 0 or seed > 2147483647:
        seed = None

    # Use & provide a seed, if possible, so that we can reproduce the results when needed.
    images = []
    error_message = ""
    try:
        images, generate_response = imagen_generate(
            model_name, prompt, negative_prompt, sampleImageSize, sampleCount, seed
        )
    except Exception as e:
        print(e)
        error_message = """An error occured calling the API.
      1. Check if response was not blocked based on policy violation, check if the UI behaves the same way.
      2. Try a different prompt to see if that was the problem.
      """
        error_message += "\n" + traceback.format_exc()
        # raise gr.Error(str(e))

    return images, error_message

### 定義 Gradio 範例

範例文字提示提供用於產生圖片，你也可以嘗試你自己的文字提示。


In [ ]:
examples = [
    [
        "imagegeneration@005",
        """A studio portrait of a man with a grizzly beard eating a sandwich with his hands, a dramatic skewed angled photography, film noir.""",
        "",
        "1536",
        4,
        -1,
    ],
    [
        "imagegeneration@005",
        """A mosaic-inspired portrait of a person, their features formed by a collection of small, colourful tiles.""",
        "",
        "1536",
        4,
        -1,
    ],
    [
        "imagegeneration@005",
        """A modern house on a coastal cliff, sunset, reflections in the water, bright stylized, architectural magazine photo.""",
        "",
        "1536",
        4,
        -1,
    ],
    [
        "imagegeneration@005",
        """Isometric 3d rendering of a car driving in the countryside surrounded by trees, bright colors, puffy clouds overhead.""",
        "",
        "1536",
        4,
        -1,
    ],
    [
        "imagegeneration@005",
        """A tube of toothpaste with the words "CYMBAL" written on it, on a bathroom counter, advertisement.""",
        "",
        "1536",
        4,
        -1,
    ],
    [
        "imagegeneration@005",
        """A cup of strawberry yogurt with the word "Delicious" written on its side, sitting on a wooden tabletop. Next to the cup of yogurt is a plat with toast and a glass of orange juice.""",
        "",
        "1536",
        4,
        -1,
    ],
    [
        "imagegeneration@005",
        """A clearn minimal emblem style logo for an ice cream shop, cream background.""",
        "",
        "1536",
        4,
        -1,
    ],
    [
        "imagegeneration@005",
        """An abstract logo representing intelligence for an enterprise AI platform, "Vertex AI" written under the logo.""",
        "",
        "1536",
        4,
        -1,
    ],
    [
        "imagegeneration@002",
        """A line drawing of a duck boat tour in Boston, with a colorful background of the city.""",
        "",
        "1024",
        4,
        -1,
    ],
    [
        "imagegeneration@002",
        """A raccoon wearing formal clothes, wearing a top hat. Oil painting in the style of Vincent Van Gogh.""",
        "",
        "1024",
        4,
        -1,
    ],
]

## Gradio 界面

本節啟動一個 [Gradio Interface](https://www.gradio.app/docs/interface)，可經由公開 URL 開啟，或直接從筆記本中使用。歡迎使用不同的文字提示進行實驗以產生影像。


In [ ]:
# https://gradio.app/docs/#gallery
iface = gr.Interface(
    fn=update,
    inputs=[
        gr.Dropdown(
            label="Model Name",
            choices=["imagegeneration@002", "imagegeneration@005"],
            value="imagegeneration@005",
        ),
        gr.Textbox(
            placeholder="Try: A studio portrait of a man with a grizzly beard eating a sandwich with his hands, a dramatic skewed angled photography, film noir.",
            label="Text Prompt",
            value="A studio portrait of a man with a grizzly beard eating a sandwich with his hands, a dramatic skewed angled photography, film noir.",
        ),
        gr.Textbox(placeholder="", label="Negative Prompt", value=""),
        gr.Dropdown(label="ImageSize", choices=["256", "1024", "1536"], value="1536"),
        gr.Number(label="sampleCount", value=4),
        gr.Number(
            label="seed",
            info="Use & provide a seed, if possible, so that we can reproduce the results when needed. Integer number range: (0, 2147483647)",
            value=-1,
        ),
    ],
    outputs=[
        gr.Gallery(
            label="Generated Images",
            show_label=True,
            elem_id="gallery",
            columns=[2],
            object_fit="contain",
            height="auto",
        ),
        gr.Textbox(label="Error Messages"),
    ],
    examples=examples,
    title="Imagen",
    description="""Image generation from a text prompt. Look at [this link](https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images) for Imagen documentation.
                     """,
    allow_flagging="never",
    theme=gr.themes.Soft(),
)

### 啟動 Gradio 應用程式，開始產生圖片！


In [ ]:
# Set debug=True in Colab for live debugging
iface.launch(debug=True)

In [ ]:
# (Optional) Make your Gradio app link publicly accessible by uncommenting the line below and running this cell
# iface.launch(share=True, debug=True)